In [27]:
import sklearn
import pandas as pd
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import svm
from datetime import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
import pickle
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


import joblib

import warnings
warnings.filterwarnings('ignore')

In [28]:
df_raw = pd.read_pickle("./data.pkl")
df_raw

,bouwjaar,kilometer_stand,vermogen,prijs,is_handgeschakeld,is_benzine,upload_datum,apk
1,2008,169504.0,78,2950.0,0.0,1,24,120
2,2007,219718.0,65,1950.0,1.0,1,19,175
3,2006,199908.0,95,1895.0,1.0,1,19,211
4,2008,138191.0,65,2740.0,1.0,1,19,373
5,2006,139057.0,65,2950.0,1.0,1,19,273
...,...,...,...,...,...,...,...,...
1124,2011,225902.0,84,3300.0,1.0,0,22,304
1125,2010,210697.0,84,3350.0,1.0,0,22,365
1126,2011,122681.0,84,3350.0,1.0,0,22,92
1127,2008,121560.0,67,3400.0,1.0,0,22,214


In [29]:
target_names = ["titel", "bouwjaar", "kilometer_stand", "vermogen", "is_handgeschakeld", "is_benzine", "upload_datum", "apk"]

numeric_features = ['bouwjaar', "kilometer_stand", "vermogen", "upload_datum", "apk"]
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', numeric_transformer, numeric_features),
#         ('text',  CountVectorizer(), 'titel')
    ],
    remainder='passthrough')

In [30]:
df_train, df_test = train_test_split(df_raw, test_size=0.2, random_state=42)

X_train = df_train.drop(['prijs'], axis=1)
y_train = df_train[['prijs']]

X_test = df_test.drop(['prijs'], axis=1)
y_test = df_test[['prijs']]['prijs']
X_test

,bouwjaar,kilometer_stand,vermogen,is_handgeschakeld,is_benzine,upload_datum,apk
59,2010,198676.0,84,1.0,0,41,141
453,2018,46883.0,90,1.0,0,22,141
616,2007,198000.0,65,1.0,1,22,23
585,2007,214000.0,77,1.0,1,22,134
53,2008,78387.0,78,1.0,1,21,347
...,...,...,...,...,...,...,...
346,2010,218547.0,84,1.0,0,22,141
545,2014,61634.0,101,1.0,1,22,175
886,2011,79612.0,84,1.0,0,22,141
697,2007,177374.0,77,1.0,1,22,164


In [31]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [32]:
def evaluate_model(y_test, y_pred):
    print("Mean absolute error:\t", int(mean_absolute_error(y_test, y_pred)))
    print("Mean squared error:\t", int(mean_squared_error(y_test, y_pred)))

In [33]:
dtree = tree.DecisionTreeRegressor()
dtree.fit(X_train, y_train)
y_pred = dtree.predict(X_test)
evaluate_model(y_test, y_pred)

Mean absolute error:	 679
Mean squared error:	 986789


In [34]:
random_forest = RandomForestRegressor(n_estimators=100)
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)
evaluate_model(y_test, y_pred)

Mean absolute error:	 499
Mean squared error:	 479121


In [35]:
joblib.dump(random_forest, 'RandomForestRegressor')
joblib.dump(preprocessor, 'RFPreprocessor')

['RFPreprocessor']